In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Absenteeism-data.csv')
data.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


Here we will check for any nulls in the dataset

In [3]:
data.isnull().sum()

ID                           0
Reason for Absence           0
Date                         0
Transportation Expense       0
Distance to Work             0
Age                          0
Daily Work Load Average      0
Body Mass Index              0
Education                    0
Children                     0
Pets                         0
Absenteeism Time in Hours    0
dtype: int64

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         700 non-null    int64  
 1   Reason for Absence         700 non-null    int64  
 2   Date                       700 non-null    object 
 3   Transportation Expense     700 non-null    int64  
 4   Distance to Work           700 non-null    int64  
 5   Age                        700 non-null    int64  
 6   Daily Work Load Average    700 non-null    float64
 7   Body Mass Index            700 non-null    int64  
 8   Education                  700 non-null    int64  
 9   Children                   700 non-null    int64  
 10  Pets                       700 non-null    int64  
 11  Absenteeism Time in Hours  700 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


saving a copy of the original data to have as reference

In [5]:
original=data.copy()

we are tasked with diving the reasons for absences into 4 groups:<br>
    Reason 1: codes between 1 and 14<br>
    Reason 2: codes between 15 and 17<br>
    Reason 3: codes between 18 and 21<br>
    Reason 4: codes between 22 and 28<br>
All those with reason code 0 should be left as is and are not included in the new classification<br><br>
here I do this by creating a new array to hold the new codes and will use one hot encoding to split that apart

In [6]:
newcol=[]
for i in range(data.shape[0]):
    if(data.iloc[i,1]>=1 and data.iloc[i,1]<=14):
        newcol.append(1)
    elif(data.iloc[i,1]>=15 and data.iloc[i,1]<=17):
        newcol.append(2)
    elif(data.iloc[i,1]>=18 and data.iloc[i,1]<=21):
        newcol.append(3)
    elif(data.iloc[i,1]>=22 and data.iloc[i,1]<=28):
        newcol.append(4)
    else:
        newcol.append(0)

In [7]:
newdata=pd.concat([pd.get_dummies(newcol,prefix='Reason',drop_first=True),data],axis=1)

In [8]:
newdata.head()

,Reason_1,Reason_2,Reason_3,Reason_4,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


here we will break apart the date into days and months

In [9]:
newdata['Date']=pd.to_datetime(newdata['Date'])
newdata['Day of the Week'] = newdata['Date'].dt.day
newdata['Month Value'] = newdata['Date'].dt.month

In [10]:
newdata.head()

,Reason_1,Reason_2,Reason_3,Reason_4,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Day of the Week,Month Value
0,0,0,0,1,11,26,2015-07-07,289,36,33,239.554,30,1,2,1,4,7,7
1,0,0,0,0,36,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,14,7
2,0,0,0,1,3,23,2015-07-15,179,51,38,239.554,31,1,0,0,2,15,7
3,1,0,0,0,7,7,2015-07-16,279,5,39,239.554,24,1,2,0,4,16,7
4,0,0,0,1,11,23,2015-07-23,289,36,33,239.554,30,1,2,1,2,23,7


Here is where we convert the numeric date value into what day of the week it is.<br>
Monday is 1, Tuesday is 2, ect

In [11]:
for t in range(data.shape[0]):
    #newdata.loc[t,'Day of the Week']=newdata.loc[t,'Day of the Week']%7+1
    #the above was my first attempt before learning about the pre-made option
    newdata.loc[t,'Day of the Week']=int(newdata.loc[t,'Date'].weekday())
    
newdata.head()

,Reason_1,Reason_2,Reason_3,Reason_4,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Day of the Week,Month Value
0,0,0,0,1,11,26,2015-07-07,289,36,33,239.554,30,1,2,1,4,1,7
1,0,0,0,0,36,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,1,7
2,0,0,0,1,3,23,2015-07-15,179,51,38,239.554,31,1,0,0,2,2,7
3,1,0,0,0,7,7,2015-07-16,279,5,39,239.554,24,1,2,0,4,3,7
4,0,0,0,1,11,23,2015-07-23,289,36,33,239.554,30,1,2,1,2,3,7


we can now drop all the columns we no longer need

In [12]:
newdata.drop(['Date'],axis=1,inplace=True)
newdata.drop(['ID'],axis=1,inplace=True)
newdata.drop(['Reason for Absence'],axis=1,inplace=True)
newdata.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Day of the Week,Month Value
0,0,0,0,1,289,36,33,239.554,30,1,2,1,4,1,7
1,0,0,0,0,118,13,50,239.554,31,1,1,0,0,1,7
2,0,0,0,1,179,51,38,239.554,31,1,0,0,2,2,7
3,1,0,0,0,279,5,39,239.554,24,1,2,0,4,3,7
4,0,0,0,1,289,36,33,239.554,30,1,2,1,2,3,7


Finally we will edit the values in Education such that if the value is 1 we set it to 0 and any other value is set to 1

In [13]:
for x in range(data.shape[0]):
    if(newdata.loc[x,'Education']==1):
        newdata.loc[x,'Education']=0
    else:
        newdata.loc[x,'Education']=1
newdata.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Day of the Week,Month Value
0,0,0,0,1,289,36,33,239.554,30,0,2,1,4,1,7
1,0,0,0,0,118,13,50,239.554,31,0,1,0,0,1,7
2,0,0,0,1,179,51,38,239.554,31,0,0,0,2,2,7
3,1,0,0,0,279,5,39,239.554,24,0,2,0,4,3,7
4,0,0,0,1,289,36,33,239.554,30,0,2,1,2,3,7


Now that we have done the modifications requested I will compare it to the provided solution and ensure things were done correctly. First I compared the data types and found that I needed to adjust the reason codes to the same type to compare.

In [14]:
solution = pd.read_csv('df-cleaned.csv')

In [15]:
solution.dtypes

Reason_1                       int64
Reason_2                       int64
Reason_3                       int64
Reason_4                       int64
Transportation Expense         int64
Distance to Work               int64
Age                            int64
Daily Work Load Average      float64
Body Mass Index                int64
Education                      int64
Children                       int64
Pets                           int64
Absenteeism Time in Hours      int64
Day of the Week                int64
Month Value                    int64
dtype: object

In [16]:
newdata.dtypes

Reason_1                       uint8
Reason_2                       uint8
Reason_3                       uint8
Reason_4                       uint8
Transportation Expense         int64
Distance to Work               int64
Age                            int64
Daily Work Load Average      float64
Body Mass Index                int64
Education                      int64
Children                       int64
Pets                           int64
Absenteeism Time in Hours      int64
Day of the Week                int64
Month Value                    int64
dtype: object

In [17]:
newdata['Reason_1']=newdata['Reason_1'].astype(np.int64)
newdata['Reason_2']=newdata['Reason_2'].astype(np.int64)
newdata['Reason_3']=newdata['Reason_3'].astype(np.int64)
newdata['Reason_4']=newdata['Reason_4'].astype(np.int64)

here I will use the equals function to compare each line (for some reason it doesn't work to compare the dataframes directly, still looking into why)

In [18]:
for x in range(data.shape[0]):
    if (solution[x:x].equals(newdata[x:x])):
        pass
    else:
        print(x)
    

In [19]:
solution.equals(newdata)

False